In [33]:
import pandas as pd

In [34]:
df_ball = pd.read_csv('../raw_data/IPL_Ball_by_Ball_2008_2022.csv')

In [35]:
df_match = pd.read_csv('../raw_data/IPL_Matches_2008_2022.csv')

In [36]:
df_merged = pd.merge(df_ball, df_match, on='ID')

In [37]:
df = df_merged.copy()

In [38]:
df.head(2)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,...,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,...,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon


In [39]:
df = df.drop_duplicates()
# assign the good value to overs
df["overs"] = df["overs"] + 1
# cleaning the string values
# batter
df["batter"] = [text.strip() for text in df["batter"]] # removing space
df["batter"] = [text.lower() for text in df["batter"]] # lowercase 
# bowler
df["bowler"] = [text.strip() for text in df["bowler"]]
df["bowler"] = [text.lower() for text in df["bowler"]]
# non-striker
df.rename(columns={"non-striker": "non_striker"}, inplace=True)
df["non_striker"] = [text.strip() for text in df["non_striker"]]
df["non_striker"] = [text.lower() for text in df["non_striker"]]
# extra-types
df["extra_type"] = df["extra_type"].fillna("noextra") # filling the NaN values
# player out
df["player_out"] = df["player_out"].fillna("noplayerout") # filling the NaN values
df["player_out"] = [text.strip() for text in df["player_out"]]
df["player_out"] = [text.lower() for text in df["player_out"]]
# kind
df["kind"] = df["kind"].fillna("nokind") # filling the NaN values
df["kind"] = [text.strip() for text in df["kind"]]
df["kind"] = [text.lower() for text in df["kind"]]
# fielders involved
df["fielders_involved"] = df["fielders_involved"].fillna("nofieldersinvolved") ## filling the NaN values
df["fielders_involved"] = [text.strip() for text in df["fielders_involved"]]
df["fielders_involved"] = [text.lower() for text in df["fielders_involved"]]
# batting team
df["BattingTeam"] = [text.strip() for text in df["BattingTeam"]]
df["BattingTeam"] = [text.lower() for text in df["BattingTeam"]]
# some teams change their name since 2008
team_name_dict = {'rising pune supergiants': 'rising pune supergiant', 
                  'kings xi punjab': 'punjab kings',
                  'delhi daredevils': 'delhi capitals'}
df['BattingTeam'] = df['BattingTeam'].replace(team_name_dict)
    
## cleaning data from match dataframe
# Adding the correct City for the matching Venue
df.loc[df.Venue == 'Dubai International Cricket Stadium','City'] = 'Dubai'
df.loc[df.Venue == 'Sharjah Cricket Stadium','City'] = 'Sharjah'
# Creating a dictionary of duplicated Venues so only one entry is created for each venue
venues_dict = {'Arun Jaitley Stadium, Delhi': 'Arun Jaitley Stadium',
                'Brabourne Stadium, Mumbai': 'Brabourne Stadium',
                'Dr DY Patil Sports Academy, Mumbai': 'Dr DY Patil Sports Academy',
                'Eden Gardens, Kolkata': 'Eden Gardens',  
                'M Chinnaswamy Stadium': 'M.Chinnaswamy Stadium',
                'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium',
                'MA Chidambaram Stadium, Chepauk, Chennai': 'MA Chidambaram Stadium',
                'Maharashtra Cricket Association Stadium, Pune': 'Maharashtra Cricket Association Stadium',
                'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
                'Punjab Cricket Association Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
                'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium',
                'Wankhede Stadium, Mumbai': 'Wankhede Stadium',
                'Zayed Cricket Stadium, Abu Dhabi': 'Sheikh Zayed Stadium'}
df['Venue'] = df['Venue'].replace(venues_dict)
# Two teams have changed their name over the period the dataset covers
team_name_changes = {'Kings XI Punjab': 'Punjab Kings',
                        'Delhi Daredevils': 'Delhi Capitals'}
df['Team1'] = df['Team1'].replace(team_name_changes)
# Updating the team name for Rising Pune Supergiant
team_name_dict = {'Rising Pune Supergiants': 'Rising Pune Supergiant'}
df['Team1'] = df['Team1'].replace(team_name_dict)
# Updating the other team name columns
df['Team2'] = df['Team2'].replace(team_name_changes)
df['TossWinner'] = df['TossWinner'].replace(team_name_changes)
df['WinningTeam'] = df['WinningTeam'].replace(team_name_changes)
df['Team2'] = df['Team2'].replace(team_name_dict)
df['TossWinner'] = df['TossWinner'].replace(team_name_dict)
df['WinningTeam'] = df['WinningTeam'].replace(team_name_dict)
# Identifying and removing 4 "NoResult" Values
df['WinningTeam'] = df['WinningTeam'].fillna('NoResults')
df['Player_of_Match'] = df['Player_of_Match'].fillna('NoResults')
df['SuperOver'] = df['SuperOver'].fillna('NoResults')
df.loc[df['WinningTeam'].isna(), 'method'] = 'NoResults'
df.loc[df['WinningTeam'].isna(), 'Margin'] = 0
# The method column held no useful information and was dropped
df = df.drop(columns='method')
# Results from margin related to Superover and have been filled with 0
df['Margin'] = df['Margin'].fillna(0)
    
# Updating dates so we are able to identify seasonal changes in data
season_dict = {'2020/21': '2020',
                '2009/10': '2010',
                '2007/08': '2008'}

df['Season'] = df['Season'].replace(season_dict)
# Selecting columns to strip
df_columns = list(df.select_dtypes(include='object').columns)
# Stripping the data for each column
for column in df_columns:
    df[column] = [text.strip().lower() for text in df[column]]
    

In [40]:
df.head(2)

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,...,TossDecision,SuperOver,WinningTeam,WonBy,Margin,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,1,1,1,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon
1,1312200,1,1,2,ybk jaiswal,mohammed shami,jc buttler,legbyes,0,1,...,bat,n,gujarat titans,wickets,7.0,hh pandya,"['ybk jaiswal', 'jc buttler', 'sv samson', 'd ...","['wp saha', 'shubman gill', 'ms wade', 'hh pan...",cb gaffaney,nitin menon


In [41]:
'''
    This is where we create the new features for the model.
    They should be:
    - team_1_win --> 1 or 0 if team 1 one the match
    - innings_total --> to be used to calculate batting avgs etc.
    - team_1_batting_average --> the average amount of runs that team1 scores in a match
    - team_2_batting_average --> the average amount of runs that team2 scores in a match
    - team_1_toss_winner --> 1 or 0 if team1 won the toss
    - match_importance --> ordinal value given to the importance of the match. Final or regular etc.
    - team_1_points_against_avg --> the average number of point opposition teams score agains team_1
    - team_2_points_against_avg --> the average number of point opposition teams score agains team_2
    - team_1_avg_mvp --> average number of times that team_1 has MVP
    - team_2_avg_mvp --> average number of times that team_2 has MVP
'''

'\n    This is where we create the new features for the model.\n    They should be:\n    - team_1_win --> 1 or 0 if team 1 one the match\n    - innings_total --> to be used to calculate batting avgs etc.\n    - team_1_batting_average --> the average amount of runs that team1 scores in a match\n    - team_2_batting_average --> the average amount of runs that team2 scores in a match\n    - team_1_toss_winner --> 1 or 0 if team1 won the toss\n    - match_importance --> ordinal value given to the importance of the match. Final or regular etc.\n    - team_1_points_against_avg --> the average number of point opposition teams score agains team_1\n    - team_2_points_against_avg --> the average number of point opposition teams score agains team_2\n    - team_1_avg_mvp --> average number of times that team_1 has MVP\n    - team_2_avg_mvp --> average number of times that team_2 has MVP\n'

In [42]:
df['Team1_Win'] = (df['Team1'] == df['WinningTeam']).astype(int)



In [43]:
df['innings_total'] = df.groupby(['ID', 'innings'])['total_run'].transform('sum')

In [44]:
df = df[(df['innings'] == 1) | (df['innings'] == 2)]

In [45]:
# Convert the innings totals into columns

# Group by 'ID' and 'innings' and calculate innings_total
innings_totals = df.groupby(['ID', 'innings'])['innings_total'].mean().unstack()
innings_totals.columns = ['Team1_innings_total', 'Team2_innings_total']

# Merge the innings_totals DataFrame back into new_df on 'ID'
df = pd.merge(df, innings_totals, left_on='ID', right_index=True)

In [46]:
df['team_batting_average'] = df.groupby('BattingTeam')['total_run'].transform('sum') / df.groupby('BattingTeam')['ID'].transform('nunique')

In [47]:
df['team_batting_average'] = df.groupby('BattingTeam')['total_run'].transform('sum') / df.groupby('BattingTeam')['ID'].transform('nunique')
# Extract the relevant columns 
selected_columns = ['ID', 'innings', 'Team1', 'Team2', 'team_batting_average', 'MatchNumber','WinningTeam', 'innings_total', 'City', 'Date', 'Venue', 'TossWinner', 'TossDecision', 'Team1Players', 'Team2Players']
new_df = df[selected_columns].copy()
new_df.head()

# Filter innings 1 (Team1's innings)
team_a_data = new_df[new_df['innings'] == 1][['ID','Team1', 'team_batting_average' ]]
team_a_data.rename(columns={'team_batting_average': 'Team_1_batting_average'}, inplace=True)

# Filter innings 2 (Team1's innings)
team_b_data = new_df[new_df['innings'] == 2][['ID', 'Team2', 'team_batting_average']]
team_b_data.rename(columns={'team_batting_average': 'Team_2_batting_average'}, inplace=True)

# Merge the data from innings 1 and innings 2 for each match ID
merged_data = pd.merge(team_a_data, team_b_data, on='ID')
# Drop duplicates based on 'ID' column and keep the first occurrence
final_data = merged_data.drop_duplicates(subset='ID', keep='first')
# Reset the index
final_data.reset_index(drop=True, inplace=True)
# drop the duplicate columns
final_data.drop(columns=["Team1", "Team2"], inplace=True)
# merge with the main df 
df = pd.merge(df, final_data, on = "ID")
# drop the useless column
df.drop(columns=['team_batting_average'], inplace=True)

/var/folders/0x/l5d3n5057_q6d5cdy8j9qmv00000gn/T/ipykernel_42984/1857926164.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data.drop(columns=["Team1", "Team2"], inplace=True)


In [48]:
df['team_1_toss_winner'] = (df['Team1'] == df['TossWinner']).astype(int)

In [49]:
def map_match_number(value):
    if isinstance(value, int) or value.isnumeric():
        return 1
    elif value in ['qualifier 2', 'eliminator', 'qualifier 1', 'qualifier', 'elimination final', '3rd place play-off', 'semi final']:
        return 2
    elif value == 'final':
        return 3
    else:
        return 0

In [50]:
# Apply the function
df['MatchImportance'] = df['MatchNumber'].apply(map_match_number)

In [51]:
# Calculate the average points scored against a each team
df['Team1_points_against_avg'] = df.groupby('Team1')['Team2_innings_total'].transform('mean')
df['Team2_points_against_avg'] = df.groupby('Team2')['Team1_innings_total'].transform('mean')

In [52]:
# Calculate the average number of times that team_1 has MVP
def get_match_winner(player_of_match, team1_players, team2_players):
    if pd.isna(player_of_match):
        return 'N/A'  
    
    if pd.isna(team1_players):
        team1_players = []  
    
    if pd.isna(team2_players):
        team2_players = [] 
    
    if player_of_match in team1_players:
        return 'Team1'
    elif player_of_match in team2_players:
        return 'Team2'
    else:
        return 'N/A'
    
df['Team_MVP'] = df.apply(lambda row: get_match_winner(row['Player_of_Match'], row['Team1Players'], row['Team2Players']), axis=1)

def replace_team_mvp_with_name(row):
    if row['Team_MVP'] == 'Team1':
        return row['Team1']
    elif row['Team_MVP'] == 'Team2':
        return row['Team2']
    else:
        return row['Team_MVP']
    
df['Team_MVP'] = df.apply(replace_team_mvp_with_name, axis=1)
team_mvp_counts = df['Team_MVP'].value_counts().reset_index()

team_mvp_counts.columns = ['Team_MVP', 'MVP_Count']

# Create the columns Team1_MVP_appearances e Team2_MVP_appearances
df = df.merge(team_mvp_counts, left_on='Team1', right_on='Team_MVP', how='left').fillna(0)
df.rename(columns={'MVP_Count': 'Team1_MVP_appearances'}, inplace=True)

df = df.merge(team_mvp_counts, left_on='Team2', right_on='Team_MVP', how='left').fillna(0)
df.rename(columns={'MVP_Count': 'Team2_MVP_appearances'}, inplace=True)

# Drop columns
df.drop(['Team_MVP_x', 'Team_MVP_y', 'Team_MVP'], axis=1, inplace=True)

# Now, let's create the average of MVP appearences
total_games_team1 = df['Team1'].value_counts()
total_games_team2 = df['Team2'].value_counts()

total_games = total_games_team2 + total_games_team1

df['Team1_MVP_average'] = df['Team1_MVP_appearances'] / total_games[df['Team1']].values
df['Team2_MVP_average'] = df['Team2_MVP_appearances'] / total_games[df['Team2']].values


In [53]:
df.drop(columns = ['Team1_MVP_appearances', 'Team2_MVP_appearances'])

,ID,innings,overs,ballnumber,batter,bowler,non_striker,extra_type,batsman_run,extras_run,...,Team1_innings_total,Team2_innings_total,Team_1_batting_average,Team_2_batting_average,team_1_toss_winner,MatchImportance,Team1_points_against_avg,Team2_points_against_avg,Team1_MVP_average,Team2_MVP_average
0,1312200,1,1,1,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,130.0,133.0,155.214660,166.437500,1,3,152.938369,168.048913,0.508619,0.688241
1,1312200,1,1,2,ybk jaiswal,mohammed shami,jc buttler,legbyes,0,1,...,130.0,133.0,155.214660,166.437500,1,3,152.938369,168.048913,0.508619,0.688241
2,1312200,1,1,3,jc buttler,mohammed shami,ybk jaiswal,noextra,1,0,...,130.0,133.0,155.214660,166.437500,1,3,152.938369,168.048913,0.508619,0.688241
3,1312200,1,1,4,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,130.0,133.0,155.214660,166.437500,1,3,152.938369,168.048913,0.508619,0.688241
4,1312200,1,1,5,ybk jaiswal,mohammed shami,jc buttler,noextra,0,0,...,130.0,133.0,155.214660,166.437500,1,3,152.938369,168.048913,0.508619,0.688241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225597,335982,2,15,5,p kumar,i sharma,sb joshi,legbyes,0,1,...,222.0,82.0,153.206278,155.526549,1,1,149.992952,162.264327,0.492489,0.511228
225598,335982,2,15,6,sb joshi,i sharma,p kumar,noextra,1,0,...,222.0,82.0,153.206278,155.526549,1,1,149.992952,162.264327,0.492489,0.511228
225599,335982,2,15,7,p kumar,i sharma,sb joshi,noextra,0,0,...,222.0,82.0,153.206278,155.526549,1,1,149.992952,162.264327,0.492489,0.511228
225600,335982,2,16,1,sb joshi,lr shukla,p kumar,wides,0,1,...,222.0,82.0,153.206278,155.526549,1,1,149.992952,162.264327,0.492489,0.511228


In [54]:
df.shape


(225602, 49)

In [55]:
df = df_merged.copy()

In [56]:
df['Team1_Win'] = (df['Team1'] == df['WinningTeam']).astype(int)
# create the innings total
df['innings_total'] = df.groupby(['ID', 'innings'])['total_run'].transform('sum')
df = df[(df['innings'] == 1) | (df['innings'] == 2)]
# Convert the innings totals into columns

# Group by 'ID' and 'innings' and calculate innings_total
innings_totals = df.groupby(['ID', 'innings'])['innings_total'].mean().unstack()
innings_totals.columns = ['Team1_innings_total', 'Team2_innings_total']

# Merge the innings_totals DataFrame back into new_df on 'ID'
df = pd.merge(df, innings_totals, left_on='ID', right_index=True)
df['team_batting_average'] = df.groupby('BattingTeam')['total_run'].transform('sum') / df.groupby('BattingTeam')['ID'].transform('nunique')
# Extract the relevant columns 
selected_columns = ['ID', 'innings', 'Team1', 'Team2', 'team_batting_average', 'MatchNumber','WinningTeam', 'innings_total', 'City', 'Date', 'Venue', 'TossWinner', 'TossDecision', 'Team1Players', 'Team2Players']
new_df = df[selected_columns].copy()
new_df.head()

# Filter innings 1 (Team1's innings)
team_a_data = new_df[new_df['innings'] == 1][['ID','Team1', 'team_batting_average' ]]
team_a_data.rename(columns={'team_batting_average': 'Team_1_batting_average'}, inplace=True)

# Filter innings 2 (Team1's innings)
team_b_data = new_df[new_df['innings'] == 2][['ID', 'Team2', 'team_batting_average']]
team_b_data.rename(columns={'team_batting_average': 'Team_2_batting_average'}, inplace=True)

# Merge the data from innings 1 and innings 2 for each match ID
merged_data = pd.merge(team_a_data, team_b_data, on='ID')
# Drop duplicates based on 'ID' column and keep the first occurrence
final_data = merged_data.drop_duplicates(subset='ID', keep='first')
# Reset the index
final_data.reset_index(drop=True, inplace=True)
# drop the duplicate columns
final_data.drop(columns=["Team1", "Team2"], inplace=True)
# merge with the main df 
df = pd.merge(df, final_data, on = "ID")
# drop the useless column
df.drop(columns=['team_batting_average'], inplace=True)
# new binary column for the toss winner
df['team_1_toss_winner'] = (df['Team1'] == df['TossWinner']).astype(int)
# define a function for a new column about the match importance
def map_match_number(value):
    if isinstance(value, int) or value.isnumeric():
        return 1
    elif value in ['qualifier 2', 'eliminator', 'qualifier 1', 'qualifier', 'elimination final', '3rd place play-off', 'semi final']:
        return 2
    elif value == 'final':
        return 3
    else:
        return 0
    
# Apply the function
df['MatchImportance'] = df['MatchNumber'].apply(map_match_number)
# Calculate the average points scored against a each team
df['Team1_points_against_avg'] = df.groupby('Team1')['Team2_innings_total'].transform('mean')
df['Team2_points_against_avg'] = df.groupby('Team2')['Team1_innings_total'].transform('mean')
# Calculate the average number of times that team_1 has MVP
def get_match_winner(player_of_match, team1_players, team2_players):
    if pd.isna(player_of_match):
        return 'N/A'  
            
    if pd.isna(team1_players):
        team1_players = []  
            
    if pd.isna(team2_players):
        team2_players = [] 
            
    if player_of_match in team1_players:
        return 'Team1'
    elif player_of_match in team2_players:
        return 'Team2'
    else:
        return 'N/A'
    
df['Team_MVP'] = df.apply(lambda row: get_match_winner(row['Player_of_Match'], row['Team1Players'], row['Team2Players']), axis=1)

def replace_team_mvp_with_name(row):
    if row['Team_MVP'] == 'Team1':
        return row['Team1']
    elif row['Team_MVP'] == 'Team2':
        return row['Team2']
    else:
        return row['Team_MVP']
    
df['Team_MVP'] = df.apply(replace_team_mvp_with_name, axis=1)
team_mvp_counts = df['Team_MVP'].value_counts().reset_index()

team_mvp_counts.columns = ['Team_MVP', 'MVP_Count']

# Create the columns Team1_MVP_appearances e Team2_MVP_appearances
df = df.merge(team_mvp_counts, left_on='Team1', right_on='Team_MVP', how='left').fillna(0)
df.rename(columns={'MVP_Count': 'Team1_MVP_appearances'}, inplace=True)

df = df.merge(team_mvp_counts, left_on='Team2', right_on='Team_MVP', how='left').fillna(0)
df.rename(columns={'MVP_Count': 'Team2_MVP_appearances'}, inplace=True)

# Drop columns
df.drop(['Team_MVP_x', 'Team_MVP_y', 'Team_MVP'], axis=1, inplace=True)

# Now, let's create the average of MVP appearences
total_games_team1 = df['Team1'].value_counts()
total_games_team2 = df['Team2'].value_counts()

total_games = total_games_team2 + total_games_team1

df['Team1_MVP_average'] = df['Team1_MVP_appearances'] / total_games[df['Team1']].values
df['Team2_MVP_average'] = df['Team2_MVP_appearances'] / total_games[df['Team2']].values
df = df.drop(columns = ['Team1_MVP_appearances', 'Team2_MVP_appearances'])

/var/folders/0x/l5d3n5057_q6d5cdy8j9qmv00000gn/T/ipykernel_42984/3674772091.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data.drop(columns=["Team1", "Team2"], inplace=True)


In [25]:
df.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'method',
       'Player_of_Match', 'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2',
       'Team1_Win', 'innings_total', 'Team1_innings_total',
       'Team2_innings_total', 'Team_1_batting_average',
       'Team_2_batting_average', 'team_1_toss_winner', 'MatchImportance',
       'Team1_points_against_avg', 'Team2_points_against_avg',
       'Team1_MVP_average', 'Team2_MVP_average'],
      dtype='object')

In [57]:
df = df.drop_duplicates(subset='ID')

In [60]:
final_df = df[['City', 'Team1', 'Team2', 'Venue', 'TossDecision',
       'Team1_Win',
       'Team2_innings_total', 'Team_1_batting_average',
       'Team_2_batting_average', 'team_1_toss_winner', 'MatchImportance',
       'Team1_points_against_avg', 'Team2_points_against_avg',
       'Team1_MVP_average', 'Team2_MVP_average']]

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,...,Team1_innings_total,Team2_innings_total,Team_1_batting_average,Team_2_batting_average,team_1_toss_winner,MatchImportance,Team1_points_against_avg,Team2_points_against_avg,Team1_MVP_average,Team2_MVP_average
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,0,0,0,...,130.0,133.0,155.214660,166.437500,1,0,152.938369,168.048913,0.508619,0.688241
233,1312199,1,0,1,V Kohli,TA Boult,F du Plessis,0,0,0,...,157.0,161.0,155.526549,155.214660,0,0,149.992952,163.578637,0.492489,0.508619
469,1312198,1,0,1,V Kohli,Mohsin Khan,F du Plessis,0,0,0,...,207.0,193.0,155.526549,169.866667,0,0,149.992952,177.220221,0.492489,0.601412
725,1312197,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,0,0,0,...,188.0,191.0,155.214660,166.437500,0,0,152.938369,168.048913,0.508619,0.688241
977,1304116,1,0,1,PK Garg,LS Livingstone,Abhishek Sharma,0,1,0,...,157.0,160.0,155.302632,160.464286,1,1,147.350468,160.491858,0.482918,0.430240
1197,1304115,1,0,1,PP Shaw,DR Sams,DA Warner,0,0,0,...,159.0,160.0,160.698413,158.528139,0,1,153.339012,161.970925,0.571466,0.556756
1444,1304114,1,0,1,RD Gaikwad,TA Boult,DP Conway,0,2,0,...,150.0,151.0,160.500000,155.214660,1,1,153.212268,163.578637,0.594457,0.508619
1691,1304113,1,0,1,WP Saha,S Kaul,Shubman Gill,0,4,0,...,168.0,170.0,166.437500,155.526549,1,1,147.640719,167.736629,0.688241,0.492489
1928,1304112,1,0,1,Q de Kock,UT Yadav,KL Rahul,0,0,0,...,210.0,208.0,169.866667,153.206278,1,1,158.835882,162.264327,0.601412,0.511228
2177,1304111,1,0,1,Abhishek Sharma,DR Sams,PK Garg,0,0,0,...,193.0,190.0,155.302632,158.528139,0,1,147.350468,161.970925,0.482918,0.556756


In [27]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, RobustScaler


## preprocessing


In [28]:
## defining the list of categorical and numerical features 
categorical_columns = ['City', 'MatchNumber', 'Season', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision']
numerical_columns = ['TeamA_batting_average','TeamB_batting_average']
## apply the encoder, OneHotEncoder for categorical and the RobustScaler for numerical features
categorical_transformer = Pipeline([
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

numerical_transformer = Pipeline([
    ('scaler', RobustScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_columns),
        ('num', numerical_transformer, numerical_columns)
    ])
## create the pipeline 
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

best parameters : {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}